In [1]:
import random
from faker import Faker

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, dayofmonth

### Spark Setup

In [3]:
spark_jar_packages = ",".join([
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1",
    # "org.apache.hive:hive-metastore:3.1.3",
    # "org.apache.hive:hive-exec:3.1.3",
    # "org.apache.hadoop:hadoop-aws:3.3.4",
    # "com.amazonaws:aws-java-sdk-bundle:1.12.262",
])

In [4]:
spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("iceberg-hive-playground")
    .config("spark.jars.packages", spark_jar_packages)

    # Icerbeg-Hive Integration
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
    .config("spark.sql.catalog.spark_catalog.type", "hive")
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
    .config("hive.metastore.uris", "thrift://localhost:9083")

    # # S3 (MinIO Integration)
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")

    .getOrCreate()
)

25/01/10 17:31:27 WARN Utils: Your hostname, baptvit resolves to a loopback address: 127.0.1.1; using 192.168.2.129 instead (on interface wlp4s0)
25/01/10 17:31:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/baptvit/.ivy2/cache
The jars for the packages stored in: /home/baptvit/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3908ac0b-8404-4142-accd-954a938feef0;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1 in central


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


:: resolution report :: resolve 62ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-3908ac0b-8404-4142-accd-954a938feef0
	confs: [default]
	0 artifacts copied, 1 already retrieved (0kB/2ms)
25/01/10 17:31:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setL

### Dataset Generation

In [5]:
def generate_entry(faker: Faker, country_codes: list):
    return {
        "id": faker.unique.uuid4(),
        "name":  faker.name(),
        "email": faker.email(),
        "passport": faker.passport_number(),
        "country_code": random.choice(country_codes),
        "iban": faker.iban(),
        "swift": faker.swift11(),
        "created_at": faker.past_date(start_date='-90d').strftime('%Y-%m-%d')
    }

In [6]:
def generate_dataset(num: int, seed: int):
    country_codes = ['US', 'CA', 'JP', 'KR', 'FR', 'GE', 'UK', 'BR', 'AR']
    Faker.seed(seed)
    faker = Faker()
    return [generate_entry(faker, country_codes) for _ in range(num)]

In [7]:
dataset = generate_dataset(num=5, seed=739)

In [8]:
df = spark.createDataFrame(dataset)\
        .withColumn("year", year(col("created_at")))\
        .withColumn("month", month(col("created_at")))\
        .withColumn("day", dayofmonth(col("created_at")))

### Iceberg-Hive Integration

In [9]:
spark.sql("""
    CREATE DATABASE IF NOT EXISTS iceberg_raw
""")

DataFrame[]

In [10]:
df.writeTo("iceberg_raw.accounts")\
    .tableProperty("changelog.enabled", "true")\
    .tableProperty("overwriteSchema", "true")\
    .partitionedBy("year", "month")\
    .createOrReplace()

25/01/10 17:31:35 WARN HadoopTableOperations: Error reading version hint file file:/home/baptvit/Documents/github/lakehouse-labs/iceberg/warehouse/iceberg_raw/accounts/metadata/version-hint.text
java.io.FileNotFoundException: File file:/home/baptvit/Documents/github/lakehouse-labs/iceberg/warehouse/iceberg_raw/accounts/metadata/version-hint.text does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:160)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:372)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:976)
	at org.apache.iceberg.hado

In [12]:
spark.table("iceberg_raw.accounts").show()

+------------+----------+--------------------+--------------------+--------------------+----------------+---------+-----------+----+-----+---+
|country_code|created_at|               email|                iban|                  id|            name| passport|      swift|year|month|day|
+------------+----------+--------------------+--------------------+--------------------+----------------+---------+-----------+----+-----+---+
|          AR|2024-12-05|brightthomas@exam...|GB56NVYS608859440...|189b84f0-9527-45e...|  Brittany Heath|H58091059|BEBQGBVOSLL|2024|   12|  5|
|          AR|2024-12-12|donaldpierce@exam...|GB55LFTZ500270831...|b7e33adb-9bfe-465...|        Ann Cruz|T22953641|HMAQGBCSXE8|2024|   12| 12|
|          CA|2024-12-05|harrisondeanna@ex...|GB96EZYO163067760...|bef2df38-4a7a-4b7...| Destiny Jimenez|F75210547|WSHOGBQ55I9|2024|   12|  5|
|          GE|2024-12-06|derrick15@example...|GB14AYNQ551881503...|0daad7bc-25b6-446...|Cassidy Jones MD|595954695|VTHYGBZMNOI|2024|   12|  6|

### Upsert Dataset

In [ ]:
entries = [
    # Existing entries
    dataset[2], 
    dataset[4], 
    dataset[7],
    dataset[11],
    # New entries
    *generate_dataset(4, seed=1037)
]

In [ ]:
for entry in entries:
    username = entry['name'].lower().replace(" ", ".")
    entry['email'] = f"{username}@domain.com"

In [ ]:
upsert_df = spark.createDataFrame(entries)\
        .withColumn("year", year(col("created_at")))\
        .withColumn("month", month(col("created_at")))\
        .withColumn("day", dayofmonth(col("created_at")))

In [ ]:
upsert_df.show(8, truncate=False)

In [ ]:
upsert_df.createOrReplaceTempView("upsert_data")

### Upsert Strategy

In [ ]:
spark.sql("""
    MERGE INTO iceberg_raw.accounts AS target
    USING upsert_data AS source ON 
        target.id = source.id
    WHEN MATCHED THEN 
        UPDATE SET
            target.country_code = source.country_code,
            target.email = source.email,
            target.name = source.name,
            target.iban = source.iban,
            target.swift = source.swift,
            target.passport = source.passport
    WHEN NOT MATCHED THEN 
        INSERT *
""")

### Iceberg Metadata (WIP)

In [ ]:
spark.sql("""
    select 
        *
    from
        iceberg_raw.accounts.history
""").show(truncate=False)

In [ ]:
before_changes_df = spark.read.format("iceberg") \
    .option("snapshot-id", "1492144004614084996") \
    .load("iceberg_raw.accounts")

In [ ]:
after_changes_df = spark.read.format("iceberg") \
    .option("snapshot-id", "6466590935935897141") \
    .load("iceberg_raw.accounts")